In [ ]:
print('Esports scraping ------------------')

# LCK

In [16]:
import requests
import pandas as pd

team_full = ['⁠⁠Dplus KIA', '⁠⁠Hanwha Life Esports', '⁠⁠Liiv SANDBOX', '⁠⁠Nongshim RedForce', '⁠⁠T1', '⁠⁠OKSavingsBank BRION', '⁠⁠Gen.G', '⁠⁠DRX', '⁠⁠KT Rolster', '⁠⁠Kwangdong Freecs']
team_short = ['dk', 'hle', 'lsb', 'ns', 't1', 'bro', 'gen', 'drx', 'kt', 'kdf']

def table(league, url):
    
    response = requests.get(url)
    df = pd.read_html(response.text)

    df = df[15]
    df.columns = df.columns.droplevel(0)
    df.drop(['Legend',
             'Unnamed: 3_level_1',
             'Unnamed: 4_level_1',
             'Unnamed: 5_level_1',
             'Unnamed: 7_level_1',],
             axis=1, inplace=True)

    # 순위 생성 및 인덱스 재설정
    df.drop([0, 1, 2, 3], inplace=True)
    df.index = range(1, len(df)+1)
    df.index.name = ''
    df.reset_index(inplace=True)

    # 컬럼명 변경
    df.rename(columns = {'Unnamed: 1_level_1': 'TEAM',
                         'Unnamed: 2_level_1': 'W-L',
                         'Unnamed: 6_level_1': 'POINTS',},
                         inplace=True)

    for i in range(len(team_full)):
        df.replace(team_full[i], team_short[i], inplace=True)

    def path_to_image_html(path):
        return '<img src="/images/esports/lol/lck/'+ path + '-logo.png"/>'+ path

    df_1 = df.iloc[:5]
    df_2 = df.iloc[5:]
    print(df_1)
    print(df_2)
    df_1.to_html(f'./{league}-standings1.html', index=False, classes=f'{league}-standings', border=0, justify='center', formatters=dict(TEAM=path_to_image_html), escape=False)
    df_2.to_html(f'./{league}-standings2.html', index=False, classes=f'{league}-standings', border=0, justify='center', formatters=dict(TEAM=path_to_image_html), escape=False)

table('lck', 'https://lol.fandom.com/wiki/LCK/2023_Season/Summer_Season')

print('LCK standings done!')


     TEAM    W-L POINTS
0  1   dk  1 - 0     +2
1  2  kdf  1 - 0     +2
2  3  gen  1 - 0     +1
3  4   ns  1 - 0     +1
4  5   t1  1 - 0     +1
      TEAM    W-L POINTS
5   6   kt  1 - 1      0
6   7  bro  0 - 1     -1
7   8  drx  0 - 1     -2
8   9  lsb  0 - 1     -2
9  10  hle  0 - 2     -2
LCK standings done!


In [8]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
pd.set_option('mode.chained_assignment',  None)

def list_chunk(lst, n):
    return [lst[i:i+n] for i in range(0, len(lst), n)]

def matches(league, url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', class_='wikitable')
    df = pd.read_html(str(table))[0]

    df.columns = df.columns.droplevel(0)
    df.drop(['P', 'SB', 'VOD', 'Blue Roster', 'Red Roster'], axis=1, inplace=True)
    df.drop([0], inplace=True)
    df.reset_index(inplace=True, drop=True)

    titles = [span.get('title') for span in table.find_all('span') if span.get('title')]
    titles = list_chunk(titles, 5)
    teams = [a.get('title') for a in table.find_all('a') if a.get('title')]
    
    for i in range(0, len(df)):
        df['Blue'].loc[i] = teams[15*i+2]
        df['Red'].loc[i] = teams[15*i+3]
        df['Winner'].loc[i] = teams[15*i+4]

    for i in range(0, len(df)):
        df['Bans'].loc[i] = titles[4*i]
        df['Bans.1'].loc[i] = titles[4*i+1]
        df['Picks'].loc[i] = titles[4*i+2]
        df['Picks.1'].loc[i] = titles[4*i+3]

    for i in range(0, 5):
        df[f'blueBans{i+1}'] = df['Bans'].str[i]
        df[f'redBans{i+1}'] = df['Bans.1'].str[i]
        df[f'bluePicks{i+1}'] = df['Picks'].str[i]
        df[f'redPicks{i+1}'] = df['Picks.1'].str[i]

    df.drop(['Bans', 'Bans.1', 'Picks', 'Picks.1'], axis=1, inplace=True)
    #df['Blue'] = df['Blue'].astype('string')
    df=df.replace({'Dplus KIA': 'dk',
                   'Nongshim RedForce': 'ns',
                   'Hanwha Life Esports': 'hle',
                   'Liiv SANDBOX': 'lsb',
                   'T1': 't1',
                   'OKSavingsBank BRION': 'bro',
                   'Gen.G': 'gen',
                   'DRX': 'drx',
                   'KT Rolster': 'kt',
                   'Kwangdong Freecs': 'kdf'
                   })
    
    df['Game'] = 1
    for i in range(0, len(df)-1):
        if df['Blue'].loc[i] == df['Blue'].loc[i+1] or df['Blue'].loc[i] == df['Red'].loc[i+1]:
            df['Game'].loc[i] = '0'

    df.to_json(f'./lck/{league}-matches.json', orient='records')
    
    print(df)
    
matches('lck', 'https://lol.fandom.com/wiki/LCK/2023_Season/Summer_Season/Match_History')


          Date Blue  Red Winner blueBans1 redBans1 bluePicks1 redPicks1   
0   2023-06-09   t1  hle     t1        Vi    Neeko       Sion      Ornn  \
1   2023-06-09   t1  hle     t1        Vi    Neeko        Jax      Sion   
2   2023-06-09  hle   t1    hle   K'Sante    Neeko       Sion       Jax   
3   2023-06-09   kt  gen    gen   K'Sante    Neeko   Renekton       Jax   
4   2023-06-09  gen   kt    gen   Sejuani    Neeko        Jax      Ornn   
5   2023-06-09   kt  gen     kt   K'Sante    Neeko        Jax      Gnar   
6   2023-06-08  bro   ns     ns   K'Sante   Maokai     Gragas      Gnar   
7   2023-06-08   ns  bro     ns      Zeri  K'Sante       Sion    Gragas   
8   2023-06-08  bro   ns    bro   K'Sante   Maokai     Gragas    Aatrox   
9   2023-06-08  drx  kdf    kdf    Maokai    Milio       Sion      Gwen   
10  2023-06-08  kdf  drx    kdf    Lucian    Neeko    K'Sante      Sion   
11  2023-06-07  lsb   dk     dk     Annie    Neeko        Jax     Poppy   
12  2023-06-07   dk  lsb 